In [27]:
# openai key is loaded in .env file. load the environment variables
%load_ext dotenv
%dotenv


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


### Create OpenAI model
Two model - OpenAI and ChatOpenAI.
OpenAI model, the input and output are strings, while with ChatOpenAI model, the input is a sequence of messages and the output is a AIMessage.

In [20]:
from langchain_openai import OpenAI
# crate llm using openai 
llm = OpenAI(openai_api_key=getenv("OPENAI_API_KEY"))

In [22]:
res = llm.invoke("how are you?")

print(type(res))
print(res)

<class 'str'>


I am a digital assistant and do not have emotions. I am always functioning at my best and ready to assist you with any tasks or questions you may have. How can I help you today?


In [29]:
from langchain_openai import ChatOpenAI
# chatopenai model
chat_llm = ChatOpenAI(openai_api_key=getenv("OPENAI_API_KEY"))

chat_res = chat_llm.invoke("how are you?")

print(type(chat_res))
print("AI Message:= ", chat_res)
print("Content:= ", chat_res.content)


<class 'langchain_core.messages.ai.AIMessage'>
AI Message:=  content="I'm just a computer program, so I don't have feelings like humans do. But I'm here to help you with anything you need. How can I assist you today?" response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 11, 'total_tokens': 47}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-dca72145-464e-4055-8c96-b7628b683424-0'
Content:=  I'm just a computer program, so I don't have feelings like humans do. But I'm here to help you with anything you need. How can I assist you today?


In [31]:
# chatopen ai llm accept multiple messages
from langchain_core.messages import HumanMessage, SystemMessage

# chat history
# chain of messages and chat history
messages = [
    HumanMessage("Hello, my name is raghav. when you answer my questions, start addressing by my name"),
    SystemMessage("Hello raghav, how can I help you?"),
    HumanMessage("do any know steps to book flight?"),
]

res = chat_llm.invoke(messages)

print("ChatOpenAI message type:= ", type(res))
print("Response:= ", res.content)

ChatOpenAI message type:=  <class 'langchain_core.messages.ai.AIMessage'>
Response:=  Hello raghav, to book a flight, you can follow these steps:

1. Visit a flight booking website or app.
2. Enter your departure city, destination city, dates of travel, and number of passengers.
3. Browse through the available flight options and select the one that suits your preferences.
4. Enter passenger details and contact information.
5. Choose any additional services or add-ons you may need, such as seat selection or extra baggage.
6. Review your booking details, including the total cost.
7. Proceed to payment and enter your payment information.
8. Once the payment is confirmed, you will receive a booking confirmation with your flight details.

I hope this helps! Let me know if you have any other questions.


In [58]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])

prompt_value = template.invoke(
    {
        "name": "Bob",
        "user_input": "What is your name?"
    }
)
print( prompt_value)

messages=[SystemMessage(content='You are a helpful AI bot. Your name is Bob.'), HumanMessage(content='Hello, how are you doing?'), AIMessage(content="I'm doing well, thanks!"), HumanMessage(content='What is your name?')]


Output parser - parse the AIMessage using different parser

In [32]:
from langchain_core.output_parsers import StrOutputParser

# LCEL
llm_chat_str_parser = chat_llm | StrOutputParser() # LCEL

res = llm_chat_str_parser.invoke(messages)

print("ChatOpenAI message type:= ", type(res))
print("Response:= ", res)

ChatOpenAI message type:=  <class 'str'>
Response:=  Hello raghav, to book a flight, you can follow these steps:

1. Visit a flight booking website or app.
2. Enter your departure city, destination city, travel dates, and the number of passengers.
3. Browse available flights and select the one that best fits your schedule and budget.
4. Enter passenger details and payment information to complete the booking.
5. Once the booking is confirmed, you will receive a confirmation email with your flight details.

If you need any further assistance, feel free to ask, raghav.


# Prompts
A prompt for a language model is a set of instructions or input provided by a user to guide the model's response, helping it understand the context and generate relevant and coherent language-based output, such as answering questions, completing sentences, or engaging in a conversation.

In [40]:
from langchain_core.prompts import ChatPromptTemplate

# chat prompt template
template = '''You are system admin who can understand the resource utilization metrics of a server or system like cpu utilization, memory utilization, storage utilization and network utilization in different format.

Translate the metrics details in below json format: 
All the metrics values must be in exact percentage.
{{
    "cpu": "cpu utilization in percentage and of type integer",
    "memory": "memory utilization in percentage is in string with percentage",
    "storage": "storage utilization type integer",
    "network": "network utilization in percentage"
}}

system utilization metrics in unstructured format:

{resource_utilization}
'''
prompt = ChatPromptTemplate.from_template(template=template, input_args=["resource_utilization"])

llm_prompt = prompt | chat_llm

res = llm_prompt.invoke({"resource_utilization": "cpu consumption is 20%, memory is 30%, storage = 40%, network utilization is 50%"})

print("ChatOpenAI message type:= ", type(res))
print("Response:= ", res.content)
print("Content Type:= ", type(res.content))


ChatOpenAI message type:=  <class 'langchain_core.messages.ai.AIMessage'>
Response:=  {
    "cpu": 20,
    "memory": "30%",
    "storage": 40,
    "network": 50
}
Content Type:=  <class 'str'>


In [60]:
# parse the output using jsonparser 
from langchain_core.output_parsers import JsonOutputParser

llm_prompt_json_parser = llm_prompt | JsonOutputParser()

res = llm_prompt_json_parser.invoke({"resource_utilization": "cpu utilization is 20%, memory utilization is 30%, storage utilization is 40%, network utilization is 50%"})

print("ChatOpenAI message type:= ", type(res))
print("Response:= ", res)
print("CPU:= ", res['cpu'])


ChatOpenAI message type:=  <class 'dict'>
Response:=  {'cpu': 20, 'memory': '30%', 'storage': 40, 'network': 50}
CPU:=  20


Parse output to Python object

In [45]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

class ResourceUtilization(BaseModel):
    cpu: str = Field(..., description="cpu utilization in percentage")
    memory: str = Field(..., description="memory utilization in percentage")
    storage: str = Field(..., description="storage utilization in percentage")
    network: str = Field(..., description="network utilization in percentage")
    
pydantic_parser = PydanticOutputParser(pydantic_object=ResourceUtilization)

print( pydantic_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"cpu": {"title": "Cpu", "description": "cpu utilization in percentage", "type": "string"}, "memory": {"title": "Memory", "description": "memory utilization in percentage", "type": "string"}, "storage": {"title": "Storage", "description": "storage utilization in percentage", "type": "string"}, "network": {"title": "Network", "description": "network utilization in percentage", "type": "string"}}, "required": ["cpu", "memory", "storage", "network"]}
```


LLM with Pydantic Parser integration

In [59]:
# chat prompt template
template = '''You are system admin who can understand the resource unstructured utilization metrics of a server or system like cpu utilization, memory utilization, storage utilization and network utilization.
{format_instructions}
System utilization metrics in unstructured format:
{resource_utilization}
'''
prompt = ChatPromptTemplate.from_template(template=template, 
                                          input_args=["resource_utilization"], 
                                          partial_variables={"format_instructions": pydantic_parser.get_format_instructions()})

llm_prompt_json_parser = prompt | chat_llm | pydantic_parser

res = llm_prompt_json_parser.invoke({"resource_utilization": "cpu utilization is 20%, memory = 30%, storage utilization is 40%, network = 50%"})

print("ChatOpenAI message type:= ", type(res))
print("Response:= ", res)
print("CPU:= ", res.cpu)

ChatOpenAI message type:=  <class '__main__.ResourceUtilization'>
Response:=  cpu='20%' memory='30%' storage='40%' network='50%'
CPU:=  20%


ref: https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html